## Part 4: Toy generation and bias studies
With combine we can generate toy datasets from the compiled datacard workspace. Please read [this section](https://cms-analysis.github.io/HiggsAnalysis-CombinedLimit/part3/runningthetool/#toy-data-generation) in the combine manual before proceeding.

An interesting use case of toy generation is when performing bias studies. In the Higgs to two photon (Hgg) analysis, the background is fit with some functional form. However (due to the complexities of QCD) the exact form of this function is unknown. Therefore, we need to understand how our choice of background function may impact the fitted signal strength. This is performed using a bias study, which will indicate how much potential bias is present given a certain choice of functional form.

In the classical bias studies we begin by building a set of workspaces which correspond to different background function choices. In addition to the `RooExponential` constructed in Section 1, let's also try a (4th order) `RooChebychev` polynomial and a simple power law function to fit the background $m_{\gamma\gamma}$ distribution. 

You can build the different functions and the workspace with the code blocks below. Take some time to look at the blocks and understand what the code is doing. In particular notice how we have saved the data as a `RooDataHist` in the workspace. This means we are now performing **binned** maximum likelihood fits (this is useful for part 4 to speed up fitting the many toys). If the binning is sufficiently granular, then there will be no noticeable difference in the results to the **unbinned** likelihood fits. Run the script with:

In [ ]:
import ROOT
from IPython.display import Image

In [ ]:
# Open the data TTree and convert to RooDataSet
mass = ROOT.RooRealVar("CMS_hgg_mass", "CMS_hgg_mass", 125, 100, 180)
weight = ROOT.RooRealVar("weight","weight",0,0,1)

# Load the data 
f = ROOT.TFile("data_part1.root","r")
t = f.Get("data_Tag0")

# Convert to RooDataSet
data = ROOT.RooDataSet("data_Tag0", "data_Tag0", t, ROOT.RooArgSet(mass), "", "weight")

In [ ]:
# Define ranges to fit
n_bins = 80
binning = ROOT.RooFit.Binning(n_bins,100,180)
mass.setRange("loSB", 100, 115 )
mass.setRange("hiSB", 135, 180 )
mass.setRange("full", 100, 180 )
fit_range = "loSB,hiSB"

# Define the different background model pdf choices and fit to the data mass sidebands
# RooExponential
alpha = ROOT.RooRealVar("alpha", "alpha", -0.05, -0.2, 0 )
model_exp_bkg = ROOT.RooExponential("model_exp_bkg_Tag0", "model_exp_bkg_Tag0", mass, alpha )
# Fit model to data sidebands
model_exp_bkg.fitTo( data, ROOT.RooFit.Range(fit_range), ROOT.RooFit.Minimizer("Minuit2","minimize"),ROOT.RooFit.SumW2Error(True), ROOT.RooFit.PrintLevel(-1) )

# RooChebychev polynomial: 4th order
poly_1 = ROOT.RooRealVar("poly_1","T1 of chebychev polynomial", 0.01, -4, 4)
poly_2 = ROOT.RooRealVar("poly_2","T2 of chebychev polynomial", 0.01, -4, 4)
poly_3 = ROOT.RooRealVar("poly_3","T3 of chebychev polynomial", 0.01, -4, 4)
poly_4 = ROOT.RooRealVar("poly_4","T4 of chebychev polynomial", 0.01, -4, 4)
model_poly_bkg = ROOT.RooChebychev("model_poly_bkg_Tag0", "model_poly_bkg_Tag0", mass, ROOT.RooArgList(poly_1,poly_2,poly_3,poly_4) )
# Fit model to data sidebands
model_poly_bkg.fitTo( data, ROOT.RooFit.Range(fit_range), ROOT.RooFit.Minimizer("Minuit2","minimize"),ROOT.RooFit.SumW2Error(True), ROOT.RooFit.PrintLevel(-1) )

# Power law function: using RooGenericPdf functionality
pow_1 = ROOT.RooRealVar("pow_1","Exponent of power law", -3, -10, -0.0001)
model_pow_bkg = ROOT.RooGenericPdf("model_pow_bkg_Tag0", "TMath::Power(@0,@1)", ROOT.RooArgList(mass,pow_1) )
# Fit model to data sidebands
model_pow_bkg.fitTo( data, ROOT.RooFit.Range(fit_range), ROOT.RooFit.Minimizer("Minuit2","minimize"),ROOT.RooFit.SumW2Error(True), ROOT.RooFit.PrintLevel(-1) )

In [ ]:
# Define normalisation objects
# As data-driven fit we want the background model to have a freely floating yield in the final fit
norm_exp = ROOT.RooRealVar("model_exp_bkg_Tag0_norm", "Number of background events in Tag0 (exponential)", data.numEntries(), 0, 3*data.numEntries() )
norm_poly = ROOT.RooRealVar("model_poly_bkg_Tag0_norm", "Number of background events in Tag0 (polynomial)", data.numEntries(), 0, 3*data.numEntries() )
norm_pow = ROOT.RooRealVar("model_pow_bkg_Tag0_norm", "Number of background events in Tag0 (power law)", data.numEntries(), 0, 3*data.numEntries() )

# Also we want parameters of models to be floating in final fit to data
# Therefore no need to set the shape parameters of the model to be constant

In [ ]:
# Lets save the data as a RooDataHist with 320 bins between 100 and 180
mass.setBins(320)
data_hist = ROOT.RooDataHist("data_hist_Tag0", "data_hist_Tag0", mass, data )

In [ ]:
# Save the background model and data set to a RooWorkspace
w_bkg = {}
for pdf in ['exp','poly','pow']:
    f_out = ROOT.TFile("workspace_bkg_%s.root"%pdf, "RECREATE")
    w_bkg[pdf] = ROOT.RooWorkspace("workspace_bkg","workspace_bkg")
    getattr(w_bkg[pdf], "import")(data_hist)
    if pdf == 'exp':
        getattr(w_bkg[pdf], "import")(norm_exp)
        getattr(w_bkg[pdf], "import")(model_exp_bkg)
    elif pdf == 'poly':
        getattr(w_bkg[pdf], "import")(norm_poly)
        getattr(w_bkg[pdf], "import")(model_poly_bkg)
    elif pdf == 'pow':
        getattr(w_bkg[pdf], "import")(norm_pow)
        getattr(w_bkg[pdf], "import")(model_pow_bkg)
    w_bkg[pdf].Print()
    w_bkg[pdf].Write()
    f_out.Close()

In [ ]:
# Lets plot the different background functions fitted to data sidebands
can = ROOT.TCanvas()
plot = mass.frame()
# We have to be careful with the normalisation as we only fit over sidebands
data.plotOn( plot, binning, ROOT.RooFit.MarkerColor(0), ROOT.RooFit.LineColor(0) )
model_exp_bkg.plotOn( plot, ROOT.RooFit.NormRange(fit_range), ROOT.RooFit.Range("full"), ROOT.RooFit.LineColor(2), ROOT.RooFit.Name("Exponential") )
model_poly_bkg.plotOn( plot, ROOT.RooFit.NormRange(fit_range), ROOT.RooFit.Range("full"), ROOT.RooFit.LineColor(3), ROOT.RooFit.Name("Polynomial"))
model_pow_bkg.plotOn( plot, ROOT.RooFit.NormRange(fit_range), ROOT.RooFit.Range("full"), ROOT.RooFit.LineColor(4), ROOT.RooFit.Name("PowerLaw"))
data.plotOn( plot, ROOT.RooFit.CutRange(fit_range), binning )
plot.Draw()

leg = ROOT.TLegend(0.55,0.6,0.85,0.85)
leg.AddEntry("Exponential", "Exponential", "L")
leg.AddEntry("Polynomial", "Chebychev polynomial (4th order)", "L")
leg.AddEntry("PowerLaw", "Power law", "L")
leg.Draw("Same")

can.Update()
can.Draw()

The datacards for the different background model functions are saved as `datacard_part4_{pdf}.txt` where `pdf = {exp,poly,pow}`. Have a look inside the .txt files and understand what changes have been made to pick up the different functions

In [ ]:
# Let's open the datacard and take a look
for pdf in ['exp','poly','pow']:
    with open("datacard_part4_%s.txt"%pdf,"r") as f:
        lines = f.readlines()
    
    print("".join(lines))
    print("\n\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n\n")

Now compile the datacards:

In [ ]:
%%bash
for pdf in {exp,poly,pow}; do text2workspace.py datacard_part4_${pdf}.txt -m 125; done

### Bias studies
For the bias studies we want to generate ("throw") toy datasets with some choice of background function and fit back with another. The toys are thrown with a known value of the signal strength (r=1 in this example), which we will call $r_{truth}$. The fitted value of r is defined as $r_{fit}$, with some uncertainty $\sigma_{fit}$. A pull value, $P$, is calculated for each toy dataset according to,

$$ P = (r_{truth}-r_{fit})/\sigma_{fit}$$

By repeating the process for many toys we can build up a pull distribution. If there is no bias present then we would expect to obtain a normal distribution centred at 0, with a standard deviation of 1. Let's calculate the bias for our analysis.

Firstly,  we generate N=1000 toys from each of the background function choices and save them in a ROOT file. For this we use the `GenerateOnly` method of combine. We will inject signal in the toys by setting `r=1` using the `--expectSignal 1` option. 
* If time allows, repeat the bias studies with `--expectSignal 0`. This will inform us of the potential bias in the signal strength measurement given that there is no true signal.

The following commands show the example of throwing 10000 toys from the exponential function, and then fitting back with the 4th-order Chebychev polynomial. We use the `singles` algorithm to obtain a value for $r_{fit}$ and $\sigma_{fit}$ simultaneously.

In [ ]:
%%bash
combine -M GenerateOnly datacard_part4_exp.root -m 125 --freezeParameters MH -t 10000 \
-n .generate_exp --expectSignal 1 --saveToys

combine -M MultiDimFit datacard_part4_poly.root -m 125 --freezeParameters MH -t 10000 \
-n .bias_truth_exp_fit_poly --expectSignal 1 \
--toysFile higgsCombine.generate_exp.GenerateOnly.mH125.123456.root --algo singles

In [ ]:
# Now lets plot the pull distribution and fit a Gaussian to it
N_toys = 10000
r_truth = 1

truth_function = "exp"
fit_function = "poly"

name = "truth_%s_fit_%s"%(truth_function,fit_function)

# Open file with fits
f = ROOT.TFile("higgsCombine.bias_%s.MultiDimFit.mH125.123456.root"%name)
t = f.Get("limit")

hist_pull = ROOT.TH1F("pull_%s"%name, "Pull distribution: truth=%s, fit=%s"%(truth_function, fit_function), 80, -4, 4)
hist_pull.GetXaxis().SetTitle("Pull = (r_{truth}-r_{fit})/#sigma_{fit}")
hist_pull.GetYaxis().SetTitle("Entries")

for i_toy in range( N_toys ):
    # Best-fit value
    t.GetEntry(i_toy*3)
    r_fit = getattr(t, "r")

    # -1 sigma value
    t.GetEntry(i_toy*3+1)
    r_lo = getattr(t, "r")

    # +1 sigma value
    t.GetEntry(i_toy*3+2)
    r_hi = getattr(t, "r")

    diff = r_truth-r_fit
    
    # Use uncertainty depending on where mu_truth is relative to mu_fit
    if diff > 0: sigma = abs(r_hi-r_fit)
    else: sigma = abs(r_lo-r_fit)

    # Remove failed toys: really these should be investigated... but extremely small fraction here
    if sigma != 0:
        hist_pull.Fill( diff/sigma )

can = ROOT.TCanvas()
hist_pull.Draw()

# Fit Gaussian to pull distribution
ROOT.gStyle.SetOptFit(111)
hist_pull.Fit("gaus")

can.Draw()

The potential bias is defined as the (fitted) mean of the pull distribution. What is our bias value?

* What threshold do we use to define "acceptable" bias? 

From the pull definition, we see the bias value is defined relative to the total uncertainty in the signal strength (denominator of $\sigma_{fit}$). Some analyses use 0.14 as the threshold because a bias below this value would change the total uncertainty (when added in quadrature) by less than 1% (see equation below). Other analyses use 0.2 as this will change the total uncertainty by less than 2%. We should define the threshold before performing the bias study.

$$ \sqrt{ 1^2 + 0.14^2} = 1.0098 $$

* How does our bias value compare to the thresholds? If we the bias is outside the acceptable region we should account for this using a **spurious signal** method (see advanced exercises TBA).
* Repeat the bias study for each possible truth and fitted background function combinations. Do the bias values induced by the choice of background function merit adding a spurious signal component into the fit?
* What would you expect the bias value to be for a background function that does not fit the data well? Should we be worried about such functions? What test could we use to reject such functions from the study beforehand?